In [1]:
from __future__ import division
import os
import sys

import numpy as np
import scipy as sp
from scipy import stats

from Bio import SeqIO ## BioPython, to read FASTAs
import pysam

## 4A
Read in the yeast reference genome to Python (we’ve done this a few times already, so you should be good at this by now!)

In [2]:
yeastRef = SeqIO.to_dict(SeqIO.parse('sacCer3.fa', 'fasta'))

## 4D
If there are any non-reference reads, compute the genotype likelihood at that position

In [3]:
## Load BAM of BOWTIE results
aln = pysam.AlignmentFile('yeast.sorted.bam', 'rb')

In [8]:
## Look at the header
print aln.header

{'SQ': [{'LN': 230218, 'SN': 'chrI'}, {'LN': 813184, 'SN': 'chrII'}, {'LN': 316620, 'SN': 'chrIII'}, {'LN': 1531933, 'SN': 'chrIV'}, {'LN': 439888, 'SN': 'chrIX'}, {'LN': 576874, 'SN': 'chrV'}, {'LN': 270161, 'SN': 'chrVI'}, {'LN': 1090940, 'SN': 'chrVII'}, {'LN': 562643, 'SN': 'chrVIII'}, {'LN': 745751, 'SN': 'chrX'}, {'LN': 666816, 'SN': 'chrXI'}, {'LN': 1078177, 'SN': 'chrXII'}, {'LN': 924431, 'SN': 'chrXIII'}, {'LN': 784333, 'SN': 'chrXIV'}, {'LN': 1091291, 'SN': 'chrXV'}, {'LN': 948066, 'SN': 'chrXVI'}, {'LN': 85779, 'SN': 'chrM'}], 'PG': [{'PN': 'bowtie2', 'ID': 'bowtie2', 'VN': '2.1.0'}], 'HD': {'SO': 'coordinate', 'VN': '1.0'}}


In [9]:
## Make the pileup
pileup = aln.pileup()

In [21]:
## Iterate over all pileup results for SNPs only
for puc in pileup:
    chrom = puc.reference_name
    ref = yeastRef[chrom].seq[puc.reference_pos]
    reads = []
    for pu in puc.pileups:
        if pu.is_del or pu.is_refskip:
            continue
        reads.append(pu.alignment.query_sequence[pu.query_position])
    if (len(reads) == 1) and (ref in reads):
        continue   
    break ## for testing

In [27]:
reads

['A', 'C']

## 4E
Output sites where you are confident that there is a non-reference allele. Output a 6 column file: chromosome, position, reference allele, alternative allele, genotype likelihood given reference, genotype likelihood given alternative